## 一、爬取数据

In [1]:
import urllib3
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [17]:
def getPage(herf):
    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
    }
    http = urllib3.PoolManager()
    req = http.request(
        method = 'GET'
        ,url = herf
        ,headers = headers
    )
#     print(req.data)
    return req.data

def getPageNum(url):
    num = 0
    page = getPage(url)
    soup = BeautifulSoup(page)
    strong = soup.find('td', colspan='7')
    if strong:
        result = strong.get_text().split(' ')
        list_num = re.findall('[0-9]{1}', result[1])
        for i in range(len(list_num)):
            num = num*10 + int(list_num[i])
        return num
    else:
        return 0
            
def getText(url):
    for list_num in range(getPageNum(url), 0, -1):
        print(list_num)
        herf = 'http://kaijiang.zhcw.com/zhcw/html/ssq/list_' + str(list_num) + '.html'
        page = BeautifulSoup(getPage(herf))
#         print(page)
        em_list = page.find_all('em')
        div_list = page.find_all('td', {'align':'center'})
        
        text_list = []
        for i in range(9):
            text_list.append([])
        
        with open ('num.txt', 'a') as f:
            for d, div in enumerate(div_list):
                if d%5 == 0 or d%5 == 1:
                    text_list[d%5].append(div.get_text())
            text_list[0] = text_list[0][:-1]
            for e, em in enumerate(em_list):
                text_list[e%7+2].append(em.get_text())
#             print(text_list)
            for i in range(9):
                text_list[i].reverse()
            pd.DataFrame(text_list).T.to_csv('num.txt', sep = ',', mode = 'a', index = None, header = 0)

In [18]:
url = 'http://kaijiang.zhcw.com/zhcw/html/ssq/list_1.html'
getText(url)

144
143
142
141
140
139
138
137
136
135
134
133
132
131
130
129
128
127
126
125
124
123
122
121
120
119
118
117
116
115
114
113
112
111
110
109
108
107
106
105
104
103
102
101
100
99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1


## 二、预测

### 2.1 路线一：根据每个数的出现频率，出现间隔等等，预测每个数在下次是否会出现

In [2]:
data_o = pd.read_table('num.txt', sep = ',', header = None, names = ['date', 'phase', 'red1', 'red2', 'red3', 'red4', 'red5', 'red6', 'blue'], parse_dates = ['date'])
data_o.head()

,date,phase,red1,red2,red3,red4,red5,red6,blue
0,2003-02-23,2003001,10,11,12,13,26,28,11
1,2003-02-27,2003002,4,9,19,20,21,26,12
2,2003-03-02,2003003,1,7,10,23,28,32,16
3,2003-03-06,2003004,4,6,7,10,13,25,3
4,2003-03-09,2003005,4,6,15,17,30,31,16


In [30]:
data_n = data_o[data_o['date'] > '2006-02-23']
data_n.head()

,date,phase,red1,red2,red3,red4,red5,red6,blue
385,2006-02-26,2006022,2,3,4,13,16,27,13
386,2006-02-28,2006023,4,13,14,19,23,28,8
387,2006-03-02,2006024,2,7,9,11,21,27,6
388,2006-03-05,2006025,3,4,17,19,24,32,5
389,2006-03-07,2006026,1,2,18,22,29,32,3


In [5]:
# 增加辅助列

index_list = data_o.index.tolist()
data_o['index_list'] = index_list
data_o.head()

,date,phase,red1,red2,red3,red4,red5,red6,blue,index_list
0,2003-02-23,2003001,10,11,12,13,26,28,11,0
1,2003-02-27,2003002,4,9,19,20,21,26,12,1
2,2003-03-02,2003003,1,7,10,23,28,32,16,2
3,2003-03-06,2003004,4,6,7,10,13,25,3,3
4,2003-03-09,2003005,4,6,15,17,30,31,16,4


In [22]:
# 记录各数字出现的期数

appear_dict = {}
for i in range(33):
    appear_dict[i+1] = []
# appear_dict

for phase in range(data_o.shape[0]):
    tmp_list = data_o.iloc[phase].tolist()[2:-1]
    for num in range(33):
        if num+1 in tmp_list[:-1]:
            appear_dict[num+1].append(phase)
appear_dict

{1: [2,
  5,
  6,
  9,
  22,
  24,
  26,
  32,
  51,
  61,
  69,
  75,
  80,
  82,
  84,
  89,
  96,
  97,
  99,
  100,
  103,
  108,
  111,
  112,
  115,
  116,
  118,
  121,
  133,
  135,
  140,
  144,
  146,
  150,
  155,
  172,
  175,
  178,
  180,
  184,
  195,
  202,
  204,
  205,
  211,
  214,
  226,
  228,
  235,
  237,
  240,
  250,
  254,
  255,
  262,
  278,
  283,
  290,
  292,
  294,
  298,
  300,
  301,
  316,
  329,
  330,
  331,
  332,
  343,
  345,
  354,
  362,
  364,
  376,
  379,
  381,
  384,
  389,
  416,
  421,
  437,
  447,
  452,
  458,
  459,
  463,
  465,
  470,
  472,
  474,
  478,
  481,
  482,
  493,
  503,
  506,
  510,
  514,
  515,
  516,
  522,
  525,
  531,
  533,
  535,
  543,
  552,
  558,
  562,
  567,
  571,
  576,
  577,
  578,
  588,
  597,
  604,
  612,
  626,
  632,
  640,
  649,
  651,
  652,
  653,
  657,
  660,
  666,
  668,
  670,
  676,
  677,
  698,
  699,
  707,
  709,
  715,
  720,
  721,
  728,
  731,
  734,
  741,
  742,
  743,
  744

In [18]:
# 从第一个数据开始，记录频率，隔了多久，出现次数，是否出现等



{1: [2,
  5,
  6,
  9,
  22,
  24,
  26,
  32,
  51,
  61,
  69,
  75,
  80,
  82,
  84,
  89,
  96,
  97,
  99,
  100,
  103,
  108,
  111,
  112,
  115,
  116,
  118,
  121,
  133,
  135,
  140,
  144,
  146,
  150,
  155,
  172,
  175,
  178,
  180,
  184,
  195,
  202,
  204,
  205,
  211,
  214,
  226,
  228,
  235,
  237,
  240,
  250,
  254,
  255,
  262,
  278,
  283,
  290,
  292,
  294,
  298,
  300,
  301,
  316,
  329,
  330,
  331,
  332,
  343,
  345,
  354,
  362,
  364,
  376,
  379,
  381,
  384,
  389,
  416,
  421,
  437,
  447,
  452,
  458,
  459,
  463,
  465,
  470,
  472,
  474,
  478,
  481,
  482,
  493,
  503,
  506,
  510,
  514,
  515,
  516,
  522,
  525,
  531,
  533,
  535,
  543,
  552,
  558,
  562,
  567,
  571,
  576,
  577,
  578,
  588,
  597,
  604,
  612,
  626,
  632,
  640,
  649,
  651,
  652,
  653,
  657,
  660,
  666,
  668,
  670,
  676,
  677,
  698,
  699,
  707,
  709,
  715,
  720,
  721,
  728,
  731,
  734,
  741,
  742,
  743,
  744

In [23]:
sum_num = 0
for i in range(33):
    sum_num += len(appear_dict[i+1])
    
sum_num

17280

In [24]:
sum_num/data_o.shape[0]

6.0

In [25]:
# 奇偶比

def jioubi(df):
    jiou_list = []
    for i in range(df.shape[0]):
        ji = 0
        ou = 0
        for k in df.iloc[i].tolist()[2:-1]:
            if k%2 == 0:
                ji += 1
            else:
                ou += 1
        jiou_list.append(str(ji) + ':' + str(ou))
    return jiou_list


# 和值

def hezhi(df):
    hezhi_list = []
    for i in range(df.shape[0]):
        he = 0
        for k in df.iloc[i].tolist()[2:-1]:
            he += k
        hezhi_list.append(he)
    return hezhi_list


hezhi(data_o)

[100,
 99,
 101,
 65,
 103,
 88,
 99,
 76,
 104,
 65,
 94,
 104,
 114,
 75,
 117,
 150,
 110,
 113,
 86,
 127,
 128,
 78,
 110,
 93,
 71,
 122,
 98,
 115,
 111,
 85,
 130,
 126,
 99,
 99,
 41,
 132,
 116,
 96,
 114,
 64,
 89,
 60,
 91,
 97,
 88,
 79,
 139,
 131,
 78,
 100,
 100,
 110,
 132,
 101,
 115,
 129,
 147,
 129,
 64,
 78,
 109,
 90,
 90,
 75,
 117,
 94,
 99,
 145,
 107,
 61,
 94,
 74,
 102,
 114,
 138,
 97,
 103,
 127,
 129,
 136,
 102,
 122,
 90,
 75,
 74,
 107,
 66,
 113,
 144,
 48,
 123,
 90,
 128,
 84,
 109,
 106,
 105,
 82,
 67,
 99,
 129,
 138,
 84,
 72,
 109,
 102,
 79,
 105,
 93,
 95,
 96,
 89,
 115,
 111,
 106,
 75,
 43,
 120,
 91,
 93,
 72,
 61,
 99,
 125,
 106,
 81,
 126,
 124,
 113,
 111,
 85,
 90,
 90,
 112,
 115,
 85,
 90,
 107,
 107,
 68,
 88,
 67,
 105,
 119,
 126,
 134,
 90,
 90,
 98,
 124,
 122,
 101,
 125,
 148,
 110,
 75,
 89,
 99,
 128,
 94,
 132,
 89,
 100,
 127,
 94,
 83,
 90,
 100,
 104,
 112,
 133,
 148,
 70,
 113,
 112,
 70,
 109,
 134,
 128,
 120,
 85

In [17]:
data_o.shape

(2880, 9)

### 判断奇偶

In [13]:
# 做特征： 
#历史奇数百分比  历史偶数百分比  近一年奇数百分比  近一年偶数百分比  近半年奇数百分比  近半年偶数百分比  近三个月奇数百分比  近三个月偶数百分比  近一个月奇数百分比  近一个月偶数百分比
#近100次奇数百分比  近100次偶数百分比   近50次奇数百分比  近50次偶数百分比  近30次奇数百分比  近30次偶数百分比  近10次奇数百分比 近10次偶数百分比

data_o = pd.read_table('num.txt', sep = ',', header = None, names = ['date', 'phase', 'red1', 'red2', 'red3', 'red4', 'red5', 'red6', 'blue'], parse_dates = ['date'])
data_o.head()

,date,phase,red1,red2,red3,red4,red5,red6,blue
0,2003-02-23,2003001,10,11,12,13,26,28,11
1,2003-02-27,2003002,4,9,19,20,21,26,12
2,2003-03-02,2003003,1,7,10,23,28,32,16
3,2003-03-06,2003004,4,6,7,10,13,25,3
4,2003-03-09,2003005,4,6,15,17,30,31,16


In [14]:
data_o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2880 entries, 0 to 2879
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2880 non-null   datetime64[ns]
 1   phase   2880 non-null   int64         
 2   red1    2880 non-null   int64         
 3   red2    2880 non-null   int64         
 4   red3    2880 non-null   int64         
 5   red4    2880 non-null   int64         
 6   red5    2880 non-null   int64         
 7   red6    2880 non-null   int64         
 8   blue    2880 non-null   int64         
dtypes: datetime64[ns](1), int64(8)
memory usage: 202.6 KB


In [15]:
# 彩票从03年2月23号开始，2880期。大概每年153期，隔三天一期和隔四天一期。特征取成近三年的话就2495个样本


1285-1133+1

153

In [16]:
2880-386+1

2495